На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара  
1. построить свёрточные архитектуры  
2. построить различные архитектуры с RNN  
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)  
4. сдлать выводы что получилось лучше  

In [1]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D,\
        GlobalMaxPool1D, GlobalMaxPooling1D,SimpleRNN, LSTM, GRU, Masking,Concatenate,Maximum, Flatten,LayerNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  


In [2]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_valid =np.load('X_valid.npy')
y_val = np.load('y_val.npy')
training_length = X_train.shape[1]
word_count = len(np.unique(X_train))

In [2]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [3]:

exclude = set(punctuation)
morpher = MorphAnalyzer()

def removeSpecialChars(review):
     return re.sub('[^a-zа-я]', ' ', review)
    
def preprocess_text(txt):
    txt = str(txt)
    txt = txt.lower()
    txt = removeSpecialChars(txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude and len(word)>2]
    return " ".join(txt)   


df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [4]:
df_train

,id,text,class
0,0,alisachachka уезжаааааааать тоже хотеть чтобы ...,0
1,1,galyginvadim ребята девчата всё кино вот это л...,1
2,2,artem klyushin кто ненавидеть пробка ретвит,0
3,3,epupybobv хотеться котлета киевск запретный плод,1
4,4,karinekurganova yess boss босапоп есбосой боят...,1
...,...,...,...
181462,181462,классный быть новый год http lejaguxnwu,0
181463,181463,видеть человек хотеть сказать он привет ты игн...,0
181464,181464,julia styles там длинный диагноз вкратце аллер...,0
181465,181465,technoslav кто тот серенький кнопочень телефон...,1


In [5]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test =  df_test['text'].values

In [6]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = True, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)


In [7]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [3]:
def testting_model(model):
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping=EarlyStopping(monitor='val_accuracy',patience = 1)
    history = model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    callbacks=[early_stopping])
    score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
    print('\n')
    print('Test accuracy:', score[1])
    return history

In [10]:
# base line
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
tfidf = TfidfVectorizer(ngram_range=(1, 2))
x_tr = tfidf.fit_transform(df_train['text'])
x_v = tfidf.transform(df_val['text'])
lr = LogisticRegression(n_jobs=16,max_iter=2000)
lr.fit(x_tr,y_train)
accuracy_score(lr.predict(x_v), y_val)


0.7511352113917912

### Пробуем свертки

In [10]:
# 1
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,),trainable=True,
              mask_zero=True)(input_x)
x = Conv1D(filters=512,kernel_size =4, strides=1,padding='same',activation='relu')(x)
x= GlobalMaxPooling1D()(x)
x = Dense(units=64,activation = 'relu')(x)
x = Dense(units=32,activation = 'relu')(x)
out_x = Dense(units=1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 26, 128)           22839808  
_________________________________________________________________
conv1d (Conv1D)              (None, 26, 512)           262656    
_________________________________________________________________
global_max_pooling1d (Global (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33    

In [13]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 10s 44ms/step - loss: 0.5490 - accuracy: 0.7121 - val_loss: 0.5078 - val_accuracy: 0.7479
Epoch 2/10
178/178 [==============================] - 8s 42ms/step - loss: 0.3346 - accuracy: 0.8564 - val_loss: 0.5630 - val_accuracy: 0.7487
Epoch 3/10
89/89 [==============================] - 0s 3ms/step - loss: 0.6966 - accuracy: 0.7343


Test accuracy: 0.7342944145202637


In [12]:
# 1.1
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,),trainable=True,
              mask_zero=True)(input_x)
x = Conv1D(filters=512,kernel_size =4, strides=1,padding='same')(x)
x = Activation('relu')(x)
x = LayerNormalization()(x)
x= GlobalMaxPooling1D()(x)
x = Dense(units=64)(x)
x = Activation('relu')(x)
x = LayerNormalization()(x)
x = Dense(units=32)(x)
x = Activation('relu')(x)
x = LayerNormalization()(x)
out_x = Dense(units=1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 26, 128)           22839808  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 26, 512)           262656    
_________________________________________________________________
activation_1 (Activation)    (None, 26, 512)           0         
_________________________________________________________________
layer_normalization_1 (Layer (None, 26, 512)           1024      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                3283

In [13]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 10s 55ms/step - loss: 0.5791 - accuracy: 0.6936 - val_loss: 0.5034 - val_accuracy: 0.7515
Epoch 2/10
23/23 [==============================] - 0s 11ms/step - loss: 0.5493 - accuracy: 0.7453


Test accuracy: 0.7453158497810364


In [23]:
# 2
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)

x2 = Conv1D(filters=512,kernel_size =2, strides=1,padding='same')(x)
# x2 = Dropout(0.5)(x2)

x2= Activation('relu')(x2)
x2= GlobalMaxPooling1D()(x2)


x3 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x)
# x3 = Dropout(0.5)(x3)
x3= Activation('relu')(x3)
x3= GlobalMaxPooling1D()(x3)


x4 = Conv1D(filters=512,kernel_size =4, strides=1,padding='same')(x)
# x4 = Dropout(0.5)(x4)
x4= Activation('relu')(x4)
x4= GlobalMaxPooling1D()(x4)


x5 = Conv1D(filters=512,kernel_size =5, strides=1,padding='same')(x)
# x5 = Dropout(0.5)(x5)
x5= Activation('relu')(x5)
x5= GlobalMaxPooling1D()(x5)

x6 = Conv1D(filters=512,kernel_size =6, strides=1,padding='same')(x)
# x6 = Dropout(0.5)(x6)
x6= Activation('relu')(x6)
x6= GlobalMaxPooling1D()(x6)

y = Concatenate(axis=1)([x2, x2,x3,x4,x5,x6])

x = Dense(units=128)(y)
# x = Dropout(0.3)(x)
x= Activation('relu')(x)
x = Dense(units=64)(x)
# x = Dropout(0.3)(x)

x= Activation('relu')(x)
out_x = Dense(units=1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 26, 128)      22839808    input_7[0][0]                    
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 26, 512)      131584      embedding_6[0][0]                
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 26, 512)      197120      embedding_6[0][0]                
____________________________________________________________________________________________

In [24]:
testting_model(model)

Epoch 1/10
355/355 [==============================] - 27s 73ms/step - loss: 0.5301 - accuracy: 0.7287 - val_loss: 0.4988 - val_accuracy: 0.7548
Epoch 2/10
23/23 [==============================] - 1s 29ms/step - loss: 0.5922 - accuracy: 0.7446


Test accuracy: 0.7446104884147644


In [14]:
# 2.1
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)

x2 = Conv1D(filters=512,kernel_size =2, strides=1,padding='same')(x)
x2 = Dropout(0.2)(x2)
x = LayerNormalization()(x)
x2= Activation('relu')(x2)
x2= GlobalMaxPooling1D()(x2)


x3 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x)
x3 = Dropout(0.2)(x3)
x = LayerNormalization()(x)
x3= Activation('relu')(x3)
x3= GlobalMaxPooling1D()(x3)


x4 = Conv1D(filters=512,kernel_size =4, strides=1,padding='same')(x)
x4 = Dropout(0.2)(x4)
x = LayerNormalization()(x)
x4= Activation('relu')(x4)
x4= GlobalMaxPooling1D()(x4)


x5 = Conv1D(filters=512,kernel_size =5, strides=1,padding='same')(x)
x5 = Dropout(0.2)(x5)
x = LayerNormalization()(x)
x5= Activation('relu')(x5)
x5= GlobalMaxPooling1D()(x5)

x6 = Conv1D(filters=512,kernel_size =6, strides=1,padding='same')(x)
x6 = Dropout(0.2)(x6)
x = LayerNormalization()(x)
x6= Activation('relu')(x6)
x6= GlobalMaxPooling1D()(x6)

y = Concatenate(axis=1)([x2, x2,x3,x4,x5,x6])

x = Dense(units=128)(y)
x = Dropout(0.2)(x)
x = LayerNormalization()(x)
x= Activation('relu')(x)
x = Dense(units=64)(x)
x = Dropout(0.2)(x)
x = LayerNormalization()(x)

x= Activation('relu')(x)
out_x = Dense(units=1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 26, 128)      22839808    input_3[0][0]                    
__________________________________________________________________________________________________
layer_normalization_4 (LayerNor (None, 26, 128)      256         embedding_2[0][0]                
__________________________________________________________________________________________________
layer_normalization_5 (LayerNor (None, 26, 128)      256         layer_normalization_4[0][0]      
____________________________________________________________________________________________

In [15]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 33s 175ms/step - loss: 0.5942 - accuracy: 0.6697 - val_loss: 0.5158 - val_accuracy: 0.7455
Epoch 2/10
23/23 [==============================] - 1s 39ms/step - loss: 0.6365 - accuracy: 0.7398


Test accuracy: 0.7398051619529724


In [36]:
# 3
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)

x2 = Conv1D(filters=512,kernel_size =2, strides=1,padding='same')(x)
# x2 = Dropout(0.5)(x2)
x2= Activation('relu')(x2)


x3 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x2)
# x3 = Dropout(0.5)(x3)
x3= Activation('relu')(x3)


x4 = Conv1D(filters=512,kernel_size =4, strides=1,padding='same')(x3)
# x4 = Dropout(0.5)(x4)
x4= Activation('relu')(x4)


x5 = Conv1D(filters=512,kernel_size =5, strides=1,padding='same')(x4)
# x5 = Dropout(0.5)(x5)
x5= Activation('relu')(x5)

x6 = Conv1D(filters=512,kernel_size =6, strides=1,padding='same')(x5)
# x6 = Dropout(0.5)(x6)
x6= Activation('relu')(x6)
x6= GlobalMaxPooling1D()(x6)

x = Dense(units=128)(x6)
# x = Dropout(0.3)(x)
x= Activation('relu')(x)
x = Dense(units=64)(x)
# x = Dropout(0.3)(x)

x= Activation('relu')(x)
out_x = Dense(units=1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 26, 128)           22839808  
_________________________________________________________________
conv1d (Conv1D)              (None, 26, 512)           131584    
_________________________________________________________________
activation (Activation)      (None, 26, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 26, 512)           786944    
_________________________________________________________________
activation_1 (Activation)    (None, 26, 512)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 512)           104908

In [37]:
testting_model(model)

Epoch 1/10
709/709 [==============================] - 56s 77ms/step - loss: 0.5398 - accuracy: 0.7198 - val_loss: 0.5158 - val_accuracy: 0.7499
Epoch 2/10
23/23 [==============================] - 2s 55ms/step - loss: 0.6233 - accuracy: 0.7351


Test accuracy: 0.7350879311561584


In [40]:
# 4
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)

x2 = Conv1D(filters=512,kernel_size =2, strides=1,padding='same')(x)
# x2 = Dropout(0.5)(x2)
x2= Activation('relu')(x2)
x2= GlobalMaxPooling1D()(x2)

x2 = Dense(units=64)(x2)
# x2 = Dropout(0.3)(x2)
x2= Activation('relu')(x2)
x2 = Dense(units=32)(x2)
# x2 = Dropout(0.3)(x2)
out_x2 = Dense(units=2, activation='softmax',name = 'out2')(x2)
model_2 = Model(input_x,out_x2)



x3 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x)
# x3 = Dropout(0.5)(x3)
x3= Activation('relu')(x3)
x3= GlobalMaxPooling1D()(x3)

x3 = Dense(units=64)(x3)
# x3 = Dropout(0.3)(x3)
x3= Activation('relu')(x3)
x3 = Dense(units=32)(x3)
# x3 = Dropout(0.3)(x3)
out_x3 = Dense(units=2, activation='softmax',name = 'out3')(x3)
model_3 = Model(input_x,out_x3)


x4 = Conv1D(filters=512,kernel_size =4, strides=1,padding='same')(x)
# x4 = Dropout(0.5)(x4)
x4= Activation('relu')(x4)
x4= GlobalMaxPooling1D()(x4)

x4 = Dense(units=64)(x4)
# x4 = Dropout(0.3)(x4)
x4= Activation('relu')(x4)
x4 = Dense(units=32)(x4)
# x4 = Dropout(0.3)(x4)
out_x4 = Dense(units=2, activation='softmax',name = 'out4')(x4)
model_4 = Model(input_x,out_x4)


x5 = Conv1D(filters=512,kernel_size =5, strides=1,padding='same')(x)
# x5 = Dropout(0.5)(x5)
x5= Activation('relu')(x5)
x5= GlobalMaxPooling1D()(x5)

x5 = Dense(units=4)(x5)
# x5 = Dropout(0.3)(x5)
x5= Activation('relu')(x5)
x5 = Dense(units=32)(x5)
# x5 = Dropout(0.3)(x5)
out_x5 = Dense(units=2, activation='softmax',name = 'out5')(x5)
model_5 = Model(input_x,out_x5)



x6 = Conv1D(filters=512,kernel_size =6, strides=1,padding='same')(x)
# x6 = Dropout(0.5)(x6)
x6= Activation('relu')(x6)
x6= GlobalMaxPooling1D()(x6)

x6 = Dense(units=64)(x6)
# x6 = Dropout(0.3)(x6)
x6= Activation('relu')(x6)
x6 = Dense(units=32)(x6)
# x6 = Dropout(0.3)(x6)
out_x6 = Dense(units=2, activation='softmax',name = 'out6')(x6)
model_6 = Model(input_x,out_x6)

out_all = Maximum()([model_2.output, model_3.output ,model_4.output,model_5.output,model_6.output])
model_ansabl = Model(input_x,out_all,name = 'ansbl')

model_ansabl.summary()

Model: "ansbl"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 128)      22839808    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 26, 512)      131584      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 26, 512)      197120      embedding_1[0][0]                
______________________________________________________________________________________________

In [41]:
testting_model(model_ansabl)

Epoch 1/10
709/709 [==============================] - 41s 55ms/step - loss: 0.6174 - accuracy: 0.5283 - val_loss: 0.6020 - val_accuracy: 0.6299
Epoch 2/10
709/709 [==============================] - 39s 54ms/step - loss: 0.5048 - accuracy: 0.6748 - val_loss: 0.6296 - val_accuracy: 0.6552
Epoch 3/10
23/23 [==============================] - 1s 34ms/step - loss: 0.7072 - accuracy: 0.6404


Test accuracy: 0.6404355764389038


### пробуем RNN

In [19]:
# 5
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)
x = LSTM(64)(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 26, 128)           22839808  
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_15 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 33        
Total params: 22,892,385
Trainable params: 22,892,385
Non-trainable params: 0
_______________________________________________

In [20]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 7s 30ms/step - loss: 0.5548 - accuracy: 0.7116 - val_loss: 0.5360 - val_accuracy: 0.7329
Epoch 2/10
178/178 [==============================] - 5s 29ms/step - loss: 0.3784 - accuracy: 0.8331 - val_loss: 0.5374 - val_accuracy: 0.7433
Epoch 3/10
23/23 [==============================] - 0s 4ms/step - loss: 0.6397 - accuracy: 0.7302


Test accuracy: 0.7301943898200989


In [ ]:
# 5.1
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)
x = LSTM(64)(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

### пробуем смешивать



In [9]:
# 6
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)
x = Conv1D(filters=512,kernel_size =3, strides=1,padding='same',activation='relu')(x)
x = LSTM(32)(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 26, 128)           22839808  
_________________________________________________________________
conv1d (Conv1D)              (None, 26, 512)           197120    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                69760     
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17    

In [10]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 11s 47ms/step - loss: 0.5501 - accuracy: 0.7131 - val_loss: 0.5103 - val_accuracy: 0.7490
Epoch 2/10
23/23 [==============================] - 0s 11ms/step - loss: 0.5629 - accuracy: 0.7395


Test accuracy: 0.7394965291023254


In [16]:
# 7
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)
x = LSTM(32,return_sequences=True)(x)
x = Conv1D(filters=512,kernel_size =3, strides=1,padding='same',activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 26, 128)           22839808  
_________________________________________________________________
lstm_5 (LSTM)                (None, 26, 32)            20608     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 26, 512)           49664     
_________________________________________________________________
dense_9 (Dense)              (None, 26, 32)            16416     
_________________________________________________________________
dense_10 (Dense)             (None, 26, 16)            528       
_________________________________________________________________
dense_11 (Dense)             (None, 26, 1)             17  

In [17]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 9s 40ms/step - loss: 0.6625 - accuracy: 0.5636 - val_loss: 0.6514 - val_accuracy: 0.5772
Epoch 2/10
178/178 [==============================] - 7s 37ms/step - loss: 0.6095 - accuracy: 0.6139 - val_loss: 0.6568 - val_accuracy: 0.5780
Epoch 3/10
23/23 [==============================] - 0s 8ms/step - loss: 0.6775 - accuracy: 0.5760


Test accuracy: 0.5760279297828674


In [8]:
# 8
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)
x1 = Conv1D(filters=1024,kernel_size =2, strides=1,padding='same',activation='relu')(x)
x1 = GlobalMaxPooling1D()(x1)
x2 = LSTM(1024)(x)
x = Concatenate(axis=1)([x1, x2])
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 26, 128)      22839808    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 26, 1024)     263168      embedding[0][0]                  
__________________________________________________________________________________________________
global_max_pooling1d (GlobalMax (None, 1024)         0           conv1d[0][0]                     
______________________________________________________________________________________________

In [9]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 56s 298ms/step - loss: 0.5417 - accuracy: 0.7185 - val_loss: 0.5039 - val_accuracy: 0.7509
Epoch 2/10
23/23 [==============================] - 2s 96ms/step - loss: 0.5687 - accuracy: 0.7443


Test accuracy: 0.7442578077316284


In [11]:
# 9
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)
x1 = Conv1D(filters=1024,kernel_size =2, strides=1,padding='same',activation='relu')(x)
x1 = GlobalMaxPooling1D()(x1)
x2 = LSTM(1024)(x)
x = Concatenate(axis=1)([x1, x2])
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 128)      22839808    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 26, 1024)     263168      embedding_1[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 1024)         0           conv1d_1[0][0]                   
____________________________________________________________________________________________

In [12]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 54s 298ms/step - loss: 0.5404 - accuracy: 0.7165 - val_loss: 0.5119 - val_accuracy: 0.7486
Epoch 2/10
23/23 [==============================] - 2s 97ms/step - loss: 0.5754 - accuracy: 0.7396


Test accuracy: 0.739628791809082


In [4]:
# 10
input_x = Input(shape=(training_length))
x= Embedding(input_dim=word_count, output_dim=128, input_shape=(training_length,))(input_x)

x1 = Conv1D(filters=512,kernel_size =2, strides=1,padding='same')(x)
x1 = LayerNormalization()(x1)
x1 = Activation('relu')(x1)
x1 = GlobalMaxPooling1D()(x1)

x2 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x)
x2 = LayerNormalization()(x2)
x2 = Activation('relu')(x2)
x2 = GlobalMaxPooling1D()(x2)


x3 = Conv1D(filters=512,kernel_size =4, strides=1,padding='same')(x)
x3 = LayerNormalization()(x3)
x3 = Activation('relu')(x3)
x3 = GlobalMaxPooling1D()(x3)

y = LSTM(512)(x)

y1 = Conv1D(filters=512,kernel_size =2, strides=1,padding='same')(x)
y1 = LayerNormalization()(y1)
y1 = Activation('relu')(y1)
y1= LSTM(512)(y1)

y2 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x)
y2 = LayerNormalization()(y2)
y2 = Activation('relu')(y2)
y2= LSTM(512)(y2)


y3 = Conv1D(filters=512,kernel_size =3, strides=1,padding='same')(x)
y3 = LayerNormalization()(y3)
y3 = Activation('relu')(y3)
y3= LSTM(512)(y3)


x = Concatenate(axis=1)([x1, x2,x3,y,y1,y2,y3])
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
out_x = Dense(1, activation='sigmoid')(x)
model = Model(input_x,out_x)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 26, 128)      22839808    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 26, 512)      131584      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 26, 512)      197120      embedding[0][0]                  
______________________________________________________________________________________________

In [5]:
testting_model(model)

Epoch 1/10
178/178 [==============================] - 120s 636ms/step - loss: 0.5617 - accuracy: 0.7059 - val_loss: 0.5085 - val_accuracy: 0.7480
Epoch 2/10
45/45 [==============================] - 6s 118ms/step - loss: 0.5834 - accuracy: 0.7356


Test accuracy: 0.7356169819831848


Все одинаково!!! Как ни крути, больше ничего не выжать из данных. Думаю только бустинг даст небольшой прирост